In [1]:
from difflib import SequenceMatcher
import pandas as pd
import numpy as np
from tqdm import tqdm
from fuzzywuzzy import fuzz
import os

In [2]:
data_format  = 'token'

train_df = pd.read_csv(f'originial_unclean_data/{data_format}/train.tsv', sep='\t')
test_df = pd.read_csv(f'originial_unclean_data/{data_format}/test_gold.tsv', sep='\t')
dev_df = pd.read_csv(f'originial_unclean_data/{data_format}/dev.tsv', sep='\t')


In [3]:
train_df = train_df.reset_index()
test_df = test_df.reset_index()
dev_df = dev_df.reset_index() 

In [4]:
train_df['type'] = 'train'
test_df['type'] = 'test'
dev_df['type'] = 'dev'

In [5]:
assert len(dev_df[dev_df['sentence'].str.contains("\t")]) == 0

In [6]:
train_df.drop_duplicates(subset=['sentence'])

,index,sentence,label,type
0,0,ממש כואב ..... אני בוכה עם המשפחה שלא תדעו עוד...,0,train
1,1,איש יקר שלנו,0,train
2,2,כל הכבוד והמון בהצלחה,0,train
3,3,"תל חי , רובי . בכל העצב הזה היית קרן אור של ת...",0,train
4,4,נקי כפיים ובר לבב בהצלחה לך ולנו .,0,train
...,...,...,...,...
9214,9214,כבוד הנשיא הנכנס היקר איך זה שבזמן קשה כזה אנח...,1,train
9215,9215,למדינת ישראל יש נשיא סוף סוף,0,train
9216,9216,"יישר כח , אדוני הנשיא < 3",0,train
9217,9217,"איזה כיף , בהצלחה איש יקר",0,train


In [7]:
dfs = [train_df, test_df, dev_df]
dedup_df = []

In [8]:
for name, d in zip(["train_sen", "dev_sen", "test_sen"], dfs):
    d_dedup = d.drop_duplicates(subset=['sentence'])
    print(f"Before dedup {name}:", len(d))
    print(f"After dedup {name}:", len(d_dedup))
    print(f"Dedupd {name}:", len(d) - len(d_dedup))
    print(f"Precentage: {name}: {((len(d) - len(d_dedup)) / len(d)) * 100:.2f}%")
    dedup_df.append(d_dedup)

Before dedup train_sen: 9219
After dedup train_sen: 6949
Dedupd train_sen: 2270
Precentage: train_sen: 24.62%
Before dedup dev_sen: 2560
After dedup dev_sen: 2367
Dedupd dev_sen: 193
Precentage: dev_sen: 7.54%
Before dedup test_sen: 1025
After dedup test_sen: 986
Dedupd test_sen: 39
Precentage: test_sen: 3.80%


In [9]:
all_dedup = pd.concat(dedup_df)
final = all_dedup.drop_duplicates(subset=['sentence'])

In [10]:
final

,index,sentence,label,type
0,0,ממש כואב ..... אני בוכה עם המשפחה שלא תדעו עוד...,0,train
1,1,איש יקר שלנו,0,train
2,2,כל הכבוד והמון בהצלחה,0,train
3,3,"תל חי , רובי . בכל העצב הזה היית קרן אור של ת...",0,train
4,4,נקי כפיים ובר לבב בהצלחה לך ולנו .,0,train
...,...,...,...,...
1006,1006,בהצלחה רבה מגיע לו,0,dev
1014,1014,"מ""ס 1 מגיע לך להיות הנשיא שלנו",0,dev
1015,1015,שבת שלום לנשיא עם הכי הרבה קונצנזוס : ) בהצלחה...,0,dev
1017,1017,גם לכם שבת שלום ומבורכת לכם,0,dev


In [11]:
final['sentence'] = final['sentence'].apply(lambda x: x.strip("\""))

<ipython-input-11-2f2a6f3b8bae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['sentence'] = final['sentence'].apply(lambda x: x.strip("\""))


In [12]:
to_dedup = list(zip(final['sentence'], final['label'], final['index'],final['type'] ))

In [13]:
unique_comment = set()
similarity_group = {}
groups = {}
last_group = 0
for i, t1 in tqdm(enumerate(to_dedup)):
    unique = True
    for j, t2 in enumerate(to_dedup):
        if abs(len(t1[0]) - len(t2[0])) > 5 or  t2 in unique_comment or t1==t2 or t2 in groups:
            continue
        if len(t1[0]) < 7 and len(t2[0]) < 7:
            is_similar =  t1[0].strip('!,.? ') == t2[0].strip('!,.? ')
        else:
            try:
                is_similar = fuzz.partial_ratio(t1[0], t2[0]) > 95
            except ValueError:
                is_similar = SequenceMatcher(None, t1[0], t2[0]).ratio() > 0.95
        if is_similar:
            unique = False
            if t1 in groups:
                similarity_group[groups[t1]].append(t2)
                groups[t2] = groups[t1]
            else:
                groups[t1] = last_group
                groups[t2] = last_group
                similarity_group[last_group] = [t1, t2]
                last_group +=1
    if unique:
        unique_comment.add(t1)

8619it [28:15,  5.08it/s]


In [14]:
result = [{'comment': comment, 'label': label, 'index': index, 'type': set_type} for comment, label, index, set_type in unique_comment]

In [15]:
res_dedup = pd.DataFrame(result)

In [16]:
def class2int(c):
    d = {'dev': 1, 'train':0, 'test': 2}
    return d[c]
res_dedup['type'] = res_dedup['type'].apply(class2int) #
res_dedup.sort_values(['type', 'index' ])

,comment,label,index,type
872,ממש כואב ..... אני בוכה עם המשפחה שלא תדעו עוד...,0,0,0
8077,כל הכבוד והמון בהצלחה,0,2,0
5638,"תל חי , רובי . בכל העצב הזה היית קרן אור של ת...",0,3,0
1745,נקי כפיים ובר לבב בהצלחה לך ולנו .,0,4,0
546,רובי חג שמח והצלחה בבחירות לנשיאות .,0,5,0
...,...,...,...,...
5657,נשיא מדינת ישראל-תואר שרובי ריבלין ראוי לו ......,0,2551,2
3934,"בסד נשמח בשבוע . הבא עלינו לטובה לברך אותך , ...",0,2552,2
228,https://www.facebook.com/photo.php?fbid=454613...,2,2555,2
4040,שכל עם ישראל יהיה גאה בך ובמשפחתך הנהדרת .,0,2556,2


In [17]:
res_dedup_soreted = res_dedup.sort_values(['type', 'index' ])

In [18]:
res_dedup_soreted_to_split = res_dedup_soreted.reset_index(drop=True).filter(items=['comment','label'])

In [19]:
res_dedup_soreted_to_split.head()

,comment,label
0,ממש כואב ..... אני בוכה עם המשפחה שלא תדעו עוד...,0
1,כל הכבוד והמון בהצלחה,0
2,"תל חי , רובי . בכל העצב הזה היית קרן אור של ת...",0
3,נקי כפיים ובר לבב בהצלחה לך ולנו .,0
4,רובי חג שמח והצלחה בבחירות לנשיאות .,0


In [20]:
import numpy as np
def train_validate_test_split(df, train_percent=.7, validate_percent=.1, seed=None):
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[df.index[:train_end]]
    validate = df.iloc[df.index[train_end:validate_end]]
    test = df.iloc[df.index[validate_end:]]
    return train, validate, test

In [21]:
train, validate, test = train_validate_test_split(res_dedup_soreted_to_split)

In [22]:
train

,comment,label
0,ממש כואב ..... אני בוכה עם המשפחה שלא תדעו עוד...,0
1,כל הכבוד והמון בהצלחה,0
2,"תל חי , רובי . בכל העצב הזה היית קרן אור של ת...",0
3,נקי כפיים ובר לבב בהצלחה לך ולנו .,0
4,רובי חג שמח והצלחה בבחירות לנשיאות .,0
...,...,...
5921,כל הכבוד לנשיא שלנו !,0
5922,לשבעת המגייבים הקודמים : דמוקרטיה אין פירושה ...,1
5923,ישששששששששששששששש בשעה טובה איש מדהים איש אמת ...,0
5924,יחי יחי נשיא מדינת ישראל רובי רבלין.אנא מימך א...,0


In [23]:
print(len(train))
print(len(validate))
print(len(test))

5926
846
1695


In [24]:
assert len(test[test['comment'].str.contains("\t")]) ==  0

In [27]:
train.to_csv(fr'Sentiment_Data/{data_format}/train.tsv', sep='\t', index=False, encoding='utf-8')
validate.to_csv(fr'Sentiment_Data/{data_format}/dev.tsv', sep='\t', index=False, encoding='utf-8')
test.to_csv(fr'Sentiment_Data/{data_format}/test.tsv', sep='\t', index=False, encoding='utf-8')
